# DP4: Control of a Drone

Import modules.

In [1]:
import os
import time
import numpy as np
import matplotlib.pyplot as plt
import secrets
import ae353_drone
import scipy
from scipy import linalg
from scipy import signal
import sympy as sym

Create and print a seed for the random number generator so it is possible to reproduce the results.

In [2]:
seed = secrets.randbits(32)
print(seed)

3725244325


Create simulator with seed.

In [3]:
simulator = ae353_drone.Simulator(
    display=True,
    seed=seed,
)

# ================== Dynamic model ==================

Define physical parameters

In [4]:
params = {
    'm': 0.5,
    'Jx': 0.0023,
    'Jy': 0.0023,
    'Jz': 0.0040,
    'l': 0.175,
    'g': 9.81,
}

Derive equations of motion

In [5]:
# components of position (meters)
p_x, p_y, p_z = sym.symbols('p_x, p_y, p_z')

# yaw, pitch, roll angles (radians)
psi, theta, phi = sym.symbols('psi, theta, phi')

# components of linear velocity (meters / second)
v_x, v_y, v_z = sym.symbols('v_x, v_y, v_z')
v_in_body = sym.Matrix([v_x, v_y, v_z])

# components of angular velocity (radians / second)
w_x, w_y, w_z = sym.symbols('w_x, w_y, w_z')
w_in_body = sym.Matrix([w_x, w_y, w_z])

# components of net rotor torque
tau_x, tau_y, tau_z = sym.symbols('tau_x, tau_y, tau_z')

# net rotor force
f_z = sym.symbols('f_z')

# parameters
m = sym.nsimplify(params['m'])
Jx = sym.nsimplify(params['Jx'])
Jy = sym.nsimplify(params['Jy'])
Jz = sym.nsimplify(params['Jz'])
l = sym.nsimplify(params['l'])
g = sym.nsimplify(params['g'])
J = sym.diag(Jx, Jy, Jz)

# rotation matrices
Rz = sym.Matrix([[sym.cos(psi), -sym.sin(psi), 0], [sym.sin(psi), sym.cos(psi), 0], [0, 0, 1]])
Ry = sym.Matrix([[sym.cos(theta), 0, sym.sin(theta)], [0, 1, 0], [-sym.sin(theta), 0, sym.cos(theta)]])
Rx = sym.Matrix([[1, 0, 0], [0, sym.cos(phi), -sym.sin(phi)], [0, sym.sin(phi), sym.cos(phi)]])
R_body_in_world = Rz @ Ry @ Rx

# angular velocity to angular rates
ex = sym.Matrix([[1], [0], [0]])
ey = sym.Matrix([[0], [1], [0]])
ez = sym.Matrix([[0], [0], [1]])
M = sym.simplify(sym.Matrix.hstack((Ry @ Rx).T @ ez, Rx.T @ ey, ex).inv(), full=True)

# applied forces
f_in_body = R_body_in_world.T @ sym.Matrix([[0], [0], [-m * g]]) + sym.Matrix([[0], [0], [f_z]])

# applied torques
tau_in_body = sym.Matrix([[tau_x], [tau_y], [tau_z]])

# equations of motion
f = sym.Matrix.vstack(
    R_body_in_world * v_in_body,
    M * w_in_body,
    (1 / m) * (f_in_body - w_in_body.cross(m * v_in_body)),
    J.inv() * (tau_in_body - w_in_body.cross(J * w_in_body)),
)

f = sym.simplify(f, full=True)
print('The equations of motion are:')
f

The equations of motion are:


Matrix([
[v_x*cos(psi)*cos(theta) + v_y*(sin(phi)*sin(theta)*cos(psi) - sin(psi)*cos(phi)) + v_z*(sin(phi)*sin(psi) + sin(theta)*cos(phi)*cos(psi))],
[v_x*sin(psi)*cos(theta) + v_y*(sin(phi)*sin(psi)*sin(theta) + cos(phi)*cos(psi)) - v_z*(sin(phi)*cos(psi) - sin(psi)*sin(theta)*cos(phi))],
[                                                                      -v_x*sin(theta) + v_y*sin(phi)*cos(theta) + v_z*cos(phi)*cos(theta)],
[                                                                                                 (w_y*sin(phi) + w_z*cos(phi))/cos(theta)],
[                                                                                                              w_y*cos(phi) - w_z*sin(phi)],
[                                                                                  w_x + w_y*sin(phi)*tan(theta) + w_z*cos(phi)*tan(theta)],
[                                                                                                   v_y*w_z - v_z*w_y + 981*sin(theta)/100],
[   

### Linearize dynamic model

Find and check equilibirum point

In [6]:
# Suppress the use of scientific notation when printing small numbers
np.set_printoptions(suppress=True)

#create lambda function
f_num = sym.lambdify([p_x, p_y, p_z, psi, theta, phi, v_x, v_y, v_z, w_x, w_y, w_z, tau_x, tau_y, tau_z, f_z], f)

#define equilibrium values
p_xe = 0.0
p_ye = 0.0
p_ze = 0.0
psie = 0.0
thetae = 0.0
phie = 0.0
v_xe = 0.0
v_ye = 0.0
v_ze = 0.0
w_xe = 0.0
w_ye = 0.0
w_ze = 0.0
tau_xe = 0.0
tau_ye = 0.0
tau_ze = 0.0
f_ze = 9.81/2


#check if equilibirum is satisfied
check = f_num(p_xe, p_ye, p_ze, psie, thetae, phie, v_xe, v_ye, v_ze, w_xe, w_ye, w_ze, tau_xe, tau_ye, tau_ze, f_ze)
if np.all((check == 0)):
    print('Equilibirum is satisfied')
else:
    print('Check your equilibrium point')

Equilibirum is satisfied


Find A and B matrices

In [7]:
#Find A
A_num = sym.lambdify([p_x, p_y, p_z, psi, theta, phi, v_x, v_y, v_z, w_x, w_y, w_z, tau_x, tau_y, tau_z, f_z], 
                     f.jacobian([p_x, p_y, p_z, psi, theta, phi, v_x, v_y, v_z, w_x, w_y, w_z]))
A = A_num(p_xe, p_ye, p_ze, psie, thetae, phie, v_xe, v_ye, v_ze, w_xe, w_ye, w_ze, tau_xe, tau_ye, 
          tau_ze, f_ze).astype(float)
print('Your A matrix is:')
print(A, '\n')

#Find B
B_num = sym.lambdify([p_x, p_y, p_z, psi, theta, phi, v_x, v_y, v_z, w_x, w_y, w_z, tau_x, tau_y, tau_z, f_z], 
                     f.jacobian([tau_x, tau_y, tau_z, f_z]))
B = B_num(p_xe, p_ye, p_ze, psie, thetae, phie, v_xe, v_ye, v_ze, w_xe, w_ye, w_ze, tau_xe, tau_ye, 
          tau_ze, f_ze).astype(float)
print('Your B matrix is:')
print(B)

Your A matrix is:
[[ 0.    0.    0.    0.    0.    0.    1.    0.    0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.    0.    0.    0.    1.    0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.   -0.    0.   -0.    0.    1.    0.    0.    0.  ]
 [ 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    1.  ]
 [ 0.    0.    0.    0.    0.   -0.    0.    0.    0.    0.    1.   -0.  ]
 [ 0.    0.    0.    0.    0.    0.    0.    0.    0.    1.    0.    0.  ]
 [ 0.    0.    0.    0.    9.81  0.    0.    0.   -0.    0.   -0.    0.  ]
 [ 0.    0.    0.    0.    0.   -9.81 -0.    0.    0.    0.    0.   -0.  ]
 [ 0.    0.    0.    0.    0.    0.    0.   -0.    0.   -0.    0.    0.  ]
 [ 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.   -0.   -0.  ]
 [ 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.  ]] 

Your B matrix is:
[[  0.          0.          0.          0.       ]
 [  0.    

### Check for controllability

In [8]:
# Create index variable for loop
n = A.shape[0]

# Initialize W with its first column
W = B

# Create W one column at a time by iterating over i from 1 to n-1
for i in range(1, n):
    col = np.linalg.matrix_power(A, i) @ B
    W = np.block([W, col])

# Check if controllable system
print('The shape of A is')
print(A.shape , '\n')
print('The rank of W_c is')
print(np.linalg.matrix_rank(W), '\n')

if A.shape[0] == np.linalg.matrix_rank(W):
    print('The system is controllable')
else:
    print('The system is not controllable')

The shape of A is
(12, 12) 

The rank of W_c is
12 

The system is controllable


# ================== Sensor model ==================

In [9]:
g = sym.Matrix([p_x, p_y, p_z, psi])

g

Matrix([
[p_x],
[p_y],
[p_z],
[psi]])

### Linearize sensor model

Find and check equilibrium point

In [10]:
f_num = sym.lambdify([p_x, p_y, p_z, psi], g)

#define equilibrium values
p_xe = 0.0
p_ye = 0.0
p_ze = 0.0
psie = 0.0

#check if equilibirum is satisfied
check = f_num(p_xe, p_ye, p_ze, psie)
if np.all((check == 0)):
    print('Equilibirum is satisfied')
else:
    print('Check your equilibrium point')

Equilibirum is satisfied


Find C and D matrices

In [11]:
#Find C
C_num = sym.lambdify([p_x, p_y, p_z, psi, theta, phi, v_x, v_y, v_z, w_x, w_y, w_z, tau_x, tau_y, tau_z, f_z], 
                     g.jacobian([p_x, p_y, p_z, psi, theta, phi, v_x, v_y, v_z, w_x, w_y, w_z]))
C = C_num(p_xe, p_ye, p_ze, psie, thetae, phie, v_xe, v_ye, v_ze, w_xe, w_ye, w_ze, tau_xe, tau_ye, 
          tau_ze, f_ze).astype(float)
print('Your C matrix is:')
print(C, '\n')

#Find D
D_num = sym.lambdify([p_x, p_y, p_z, psi, theta, phi, v_x, v_y, v_z, w_x, w_y, w_z, tau_x, tau_y, tau_z, f_z], 
                     g.jacobian([tau_x, tau_y, tau_z, f_z]))
D = D_num(p_xe, p_ye, p_ze, psie, thetae, phie, v_xe, v_ye, v_ze, w_xe, w_ye, w_ze, tau_xe, tau_ye, 
          tau_ze, f_ze).astype(float)
print('Your D matrix is:')
print(D)

Your C matrix is:
[[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]] 

Your D matrix is:
[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]


### Check for Observability

In [12]:
# Initialize W with its first column
W = C

# Create W one row at a time by iterating over i from 1 to n-1
for i in range(1, n):
    col = C @ np.linalg.matrix_power(A, i)
    W = np.block([[W], 
                  [col]])

# Check if observable system
print('The shape of A is')
print(A.shape , '\n')
print('The rank of W_o is')
print(np.linalg.matrix_rank(W.T), '\n')

if A.shape[0] == np.linalg.matrix_rank(W.T):
    print('The system is observable')
else:
    print('The system is not observable')

The shape of A is
(12, 12) 

The rank of W_o is
12 

The system is observable


# =========== Controller and observer design: LQR ===========

Define s function to return the solution to the LQR problem:

In [13]:
def lqr(A, B ,Q, R):
    P = linalg.solve_continuous_are(A, B, Q, R)
    K = linalg.inv(R) @ B.T @ P
    return K

### Find gain matrix K
Define weights:

In [22]:
# Define Qc
Qc = 10* np.diag([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

# Define Rc
Rc = np.diag([1., 1., 1., 1.])

Find gain matrix K

In [23]:
# Using LQR solver
K = lqr(A, B, Qc, Rc)
print('Your K matrix is:')
print(K)

Your K matrix is:
[[-0.         -3.16227766 -0.          0.         -0.         17.20214873
  -0.         -4.59241875  0.          3.17476454  0.          0.        ]
 [ 3.16227766  0.         -0.         -0.         17.20214873 -0.
   4.59241875  0.          0.          0.          3.17476454 -0.        ]
 [ 0.          0.          0.          3.16227766  0.         -0.
   0.          0.          0.          0.         -0.          3.16627513]
 [-0.          0.          3.16227766  0.         -0.         -0.
  -0.          0.          3.62798534  0.          0.          0.        ]]


### Find gain matrix L
Define weights:

In [24]:
# Define Ro
Ro = np.diag([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

#Define Qo
Qo = np.diag([1., 1., 1., 1.])

Find gain matrix L

In [25]:
# Using LQR solver making the appropriate replacements
L = lqr(A.T, C.T, linalg.inv(Ro), linalg.inv(Qo)).T
print('Your L matrix is:')
print(L)

Your L matrix is:
[[ 5.2946137   0.          0.          0.        ]
 [ 0.          5.2946137   0.          0.        ]
 [ 0.          0.          1.73205081  0.        ]
 [ 0.          0.          0.          1.73205081]
 [ 1.93795015  0.          0.          0.        ]
 [ 0.         -1.93795015  0.          0.        ]
 [13.51646712  0.          0.          0.        ]
 [ 0.         13.51646712  0.          0.        ]
 [ 0.          0.          1.          0.        ]
 [ 0.         -1.          0.          0.        ]
 [ 1.          0.          0.          0.        ]
 [ 0.          0.          0.          1.        ]]


## Example of how to add a drone and run a simulation

Define a controller for the drone.

In [41]:
class Controller:
    def __init__(self):
        
        self.dt = 0.01
        self.A = A
        self.B = B
        self.C = C
        self.K = K
        self.L = L
        self.p_xe = p_xe
        self.p_ye = p_ye
        self.p_ze = p_ze
        self.psie = psie
        self.thetae = thetae
        self.phie = phie
        self.v_xe = v_xe
        self.v_ye = v_ye
        self.v_ze = v_ze
        self.w_xe = w_xe
        self.w_ye = w_ye
        self.w_ze = w_ze
        self.tau_xe = tau_xe
        self.tau_ye = tau_ye
        self.tau_ze = tau_ze
        self.f_ze = f_ze
        
        
        #keep track of state estimate
        self.variables_to_log = ['xhat']

    def get_color(self):
        return [0., 1., 0.]

    def reset(
            self,
            p_x_meas, p_y_meas, p_z_meas, # <-- measured position of drone (meters)
            yaw_meas,                     # <-- measured yaw angle of drone (radians)
        ):
        
        #initialize state estimate
        self.xhat = np.array([p_x_meas, p_y_meas, p_z_meas, yaw_meas, 0., 0., 0., 0., 0., 0., 0., 0.])

    def run(
            self,
            p_x_meas, p_y_meas, p_z_meas, # <-- measured position of drone (meters)
            yaw_meas,                     # <-- measured yaw angle of drone (radians)
            p_x_ring, p_y_ring, p_z_ring, # <-- center position of next ring (meters)
            is_last_ring,                 # <-- True if next ring is the last ring, False otherwise
            pos_others,                   # <-- 2d array of size n x 3, where n is the number
                                          #     of all *other* drones - the ith row in this array
                                          #     has the coordinates [x_i, y_i, z_i], in meters, of
                                          #     the ith other drone
        ):
        
        
        udes = np.array([0.0, 0.0, 0.0, 0.0])
        
        xdes = np.array([p_x_ring, p_y_ring, p_z_ring, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0])
        
        '''
        P_hat = self.xhat[0:3]
        P_ring = np.array([p_x_ring, p_y_ring, p_z_ring])
        r = 0.01
        pdes = P_hat + r*((P_ring-P_hat)/np.linalg.norm(P_ring-P_hat))
        xdes[0:3]=pdes
        '''
        
        u = udes -self.K @ (self.xhat - xdes)
        
        y = np.array([p_x_meas, p_y_meas, p_z_meas, yaw_meas]) - np.array([self.p_xe, self.p_ye, self.p_ze, self.psie])
        
        
        self.xhat += self.dt * (self.A@self.xhat + self.B@u - L@(self.C@self.xhat - y))
        
        tau_x = u[0] + self.tau_xe
        tau_y = u[1] + self.tau_ye
        tau_z = u[2] + self.tau_ze
        f_z = u[3] + self.f_ze

        return tau_x, tau_y, tau_z, f_z

Add the drone to the simulator. There are three arguments:

* `Controller` is the class definition
* `'my_netid'` is the name of the drone
* `'my_image.png'` is the name of an image to use as a label (can be `None`, can be a file path)

You can call this function more than once, so long as you call it with a different drone name each time. An exception will be raised if you try to add two drones of the same name to the simulator.

In [42]:
simulator.clear_drones()
simulator.add_drone(Controller, '651725086', 'Vector.png')

Reset the drone (i.e., place it at a random location in the start ring).

In [43]:
simulator.reset()
simulator.camera_droneview('651725086')

Run simulation until `max_time` is reached or until the drone reaches the finish ring.

In [44]:
simulator.run(max_time=5.)


error on run of drone 651725086 (turning it off):
Traceback (most recent call last):
  File "C:\Users\Alec\Documents\Junior_Year\AE_353\ae353-sp22\projects\04_drone\ae353_drone.py", line 798, in step
    ) = self.set_actuator_commands(
  File "C:\Users\Alec\Documents\Junior_Year\AE_353\ae353-sp22\projects\04_drone\ae353_drone.py", line 570, in set_actuator_commands
    tau_x, tau_y, tau_z, f_z = self.enforce_motor_limits(tau_x_des, tau_y_des, tau_z_des, f_z_des)
  File "C:\Users\Alec\Documents\Junior_Year\AE_353\ae353-sp22\projects\04_drone\ae353_drone.py", line 566, in enforce_motor_limits
    u = linalg.solve(self.M, s)
  File "C:\Users\Alec\anaconda3\envs\ae353\lib\site-packages\scipy\linalg\_basic.py", line 138, in solve
    b1 = atleast_1d(_asarray_validated(b, check_finite=check_finite))
  File "C:\Users\Alec\anaconda3\envs\ae353\lib\site-packages\scipy\_lib\_util.py", line 287, in _asarray_validated
    a = toarray(a)
  File "C:\Users\Alec\anaconda3\envs\ae353\lib\site-packages

C:\Users\Alec\AppData\Local\Temp\ipykernel_2200\707760546.py:73: RuntimeWarning: overflow encountered in matmul
  self.xhat += self.dt * (self.A@self.xhat + self.B@u - L@(self.C@self.xhat - y))
C:\Users\Alec\AppData\Local\Temp\ipykernel_2200\707760546.py:68: RuntimeWarning: invalid value encountered in matmul
  u = udes -self.K @ (self.xhat - xdes)
C:\Users\Alec\AppData\Local\Temp\ipykernel_2200\707760546.py:73: RuntimeWarning: invalid value encountered in matmul
  self.xhat += self.dt * (self.A@self.xhat + self.B@u - L@(self.C@self.xhat - y))


## Example of how to get and plot results

Assume that we already did two things:

* Added a drone with the name `my_netid` to the simulator
* Ran the simulation

Get result.

In [ ]:
(
    did_it_fail,
    did_it_finish,
    what_time_did_it_finish,
) = simulator.get_result('651725086')

Show result.

In [ ]:
if did_it_fail:
    print('The drone failed before finishing.')
elif did_it_finish:
    print(f'The drone finished at time {what_time_did_it_finish}')
else:
    print('The drone did not finish (yet).')

Get data.

In [ ]:
data = simulator.get_data('651725086')

Plot results.

In [ ]:
# Create a figure with subplots that all share the same x-axis
fig, (ax_pos, ax_ori, ax_act) = plt.subplots(3, 1, figsize=(12, 12), sharex=True)

# Position
ax_pos.plot(data['t'], data['p_x'], label='x (m)', linewidth=3, color='C0')
ax_pos.plot(data['t'], data['p_y'], label='y (m)', linewidth=3, color='C1')
ax_pos.plot(data['t'], data['p_z'], label='z (m)', linewidth=3, color='C2')
ax_pos.plot(data['t'], data['p_x_meas'], '.', label='measured x (m)', markersize=3, color='C0')
ax_pos.plot(data['t'], data['p_y_meas'], '.', label='measured y (m)', markersize=3, color='C1')
ax_pos.plot(data['t'], data['p_z_meas'], '.', label='measured z (m)', markersize=3, color='C2')
ax_pos.grid()
ax_pos.legend(fontsize=16, bbox_to_anchor=(1., 1.))
ax_pos.tick_params(labelsize=14)

# Orientation
ax_ori.plot(data['t'], data['yaw'], label='yaw (rad)', linewidth=3)
ax_ori.plot(data['t'], data['pitch'], label='pitch (rad)', linewidth=3)
ax_ori.plot(data['t'], data['roll'], label='roll (rad)', linewidth=3)
ax_ori.plot(data['t'], data['yaw_meas'], '.', label='measured yaw (rad)', markersize=3, color='C0')
ax_ori.grid()
ax_ori.legend(fontsize=16, bbox_to_anchor=(1., 1.))
ax_ori.tick_params(labelsize=14)

# Actuator commands
ax_act.plot(data['t'], data['tau_x'], label='tau_x (N-m)', linewidth=4)
ax_act.plot(data['t'], data['tau_x_cmd'], '--', label='commanded tau_x (N-m)', linewidth=4)
ax_act.plot(data['t'], data['tau_y'], label='tau_y (N-m)', linewidth=4)
ax_act.plot(data['t'], data['tau_y_cmd'], '--', label='commanded tau_y (N-m)', linewidth=4)
ax_act.plot(data['t'], data['tau_z'], label='tau_z (N-m)', linewidth=4)
ax_act.plot(data['t'], data['tau_z_cmd'], '--', label='commanded tau_z (N-m)', linewidth=4)
ax_act.plot(data['t'], data['f_z'] - (0.5 * 9.81), label='f_z - m g (N)', linewidth=4) # <-- FIXME
ax_act.plot(data['t'], data['f_z_cmd'] - (0.5 * 9.81), '--', label='commanded f_z - m g (N)', linewidth=4) # <-- FIXME
ax_act.grid()
ax_act.legend(fontsize=16, bbox_to_anchor=(1., 1.))
ax_act.tick_params(labelsize=14)
# ax_act.set_ylim(-10, 10) # <-- FIXME

# Set shared x-axis properties
ax_act.set_xlabel('time (s)', fontsize=20)
ax_act.set_xlim([data['t'][0], data['t'][-1]])

# Make the arrangement of subplots look nice
fig.tight_layout()
plt.show()

Save the figure (without transparent background).

In [ ]:
fig.savefig('my_figure.png', facecolor='white', transparent=False)

Plot histogram of controller run times.

In [ ]:
plt.hist(data['run_time'])
plt.ticklabel_format(style='scientific', scilimits=(0, 0), axis='x')
plt.tick_params(labelsize=14)
plt.xlabel('run time (s)', fontsize=14)
plt.ylabel('count', fontsize=14)
plt.tight_layout()
plt.show()

Note that it is required to also plot state estimates (in comparison to states) in order to check that your observer is working - no example is provided here, since no state estimates are being generated by the template `Controller` class.

## Example of how to load drones from the "students" directory

Load drones from `students` directory.

In [ ]:
failures = simulator.load_drones()

Reset drones (places drones at random positions within the start ring).

In [ ]:
simulator.reset()

Run simulation for `max_time` seconds or until all drones finish.

In [ ]:
simulator.run(max_time=5.0)

## Example of how to show results, change views, keep running

Show results (so far).

In [ ]:
simulator.show_results()

View from behind a particular drone (assuming one exists that is named `template`).

In [ ]:
simulator.camera_droneview('template')

Continue running simulation from same point for `max_time` seconds or until all drones finish.

In [ ]:
simulator.run(max_time=5.0)

View from finish ring instead of from start ring.

In [ ]:
simulator.camera_finishview()

Continue running simulation from same point for `max_time` seconds or until all drones finish.

In [ ]:
simulator.run(max_time=5.0)

Show results (so far).

In [ ]:
simulator.show_results()

## Example of how to clear and reload drones

Clear all drones.

In [ ]:
simulator.clear_drones()

Load drones again from `students` directory.

In [ ]:
simulator.load_drones()

Reset drones (places drones at random positions within the start ring).

In [ ]:
simulator.reset()

Run simulation until `max_time` is reached or until all drones finish.

In [ ]:
simulator.run(max_time=5.0)

## Example of how to rearrange the rings

Rearrange the rings.

In [ ]:
simulator.move_rings()

Reset drones (places drones at random positions within the start ring).

In [ ]:
simulator.reset()

Run simulation until `max_time` is reached or until all drones finish.

In [ ]:
simulator.run(max_time=5.0)

## Example of how to take a snapshot

View from behind a particular drone.

In [ ]:
simulator.camera_droneview('template')

Display a snapshot.

In [ ]:
# Get snapshot as 480 x 480 x 4 numpy array of RGBA values
rgba = simulator.snapshot()

# Display snapshot
plt.figure(figsize=(9, 9))
plt.imshow(rgba)

Save the snapshot.

In [ ]:
plt.imsave('my_snapshot.png', rgba)

## Example of how to record a movie

Reset drones (places drones at random positions within the start ring).

In [ ]:
simulator.reset()

View from behind a particular drone.

In [ ]:
simulator.camera_droneview('template')

Run simulation until `max_time` is reached or until all drones finish, saving movie to `my_video.mp4`.

In [ ]:
simulator.run(max_time=1.0, video_filename='my_video.mp4')

## Example of how to allow print statements and long run times

By default, your controller will fail if either of the following conditions is met:

* It prints something to `stdout`.
* It exceeds the maximum runtime of `1` second for `init`, `1` second for `reset`, and `5e-3` seconds for `run`.

You can relax these rules for the purpose of development if you want, as follows.

In [ ]:
simulator.set_rules(
    error_on_print=False,    # <-- allow print statements in controller code
    error_on_timeout=False,  # <-- allow long runtimes in controller code
)